In [1]:
import face_recognition
import os
import cv2
import numpy as np

In [2]:
KNOWN_FACES_DIR = 'known_faces'
UNKNOWN_FACES_DIR = 'unknown_faces'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2

In [3]:
print('Loading known faces...')
known_faces = []
known_names = []

Loading known faces...


In [4]:
for name in os.listdir(KNOWN_FACES_DIR):

    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):

        # Load an image
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        
        # Get 128-dimension face encoding
        # Always returns a list of found faces, for this purpose we take first face only (assuming one face per image as you can't be twice on one image)
        encoding = face_recognition.face_encodings(image)
        if len(encoding) > 0:
             biden_encoding = encoding[0]
             known_faces.append(np.array(biden_encoding).reshape(128))
             known_names.append(name)
        else:
            pass 
            
        

        # Append encodings and name
       

In [5]:
len(known_faces)

5

In [6]:
known_names


['ajay', 'ajay', 'ajay', 'tavva', 'tavva']

In [26]:
print(np.array(known_faces[0]).reshape(128))

[-1.96856856e-01  3.65737937e-02  7.90140480e-02 -9.71663976e-04
 -4.45016846e-02 -1.00752905e-01 -4.03938219e-02 -1.67211443e-02
  1.00207657e-01 -9.63001773e-02  1.85404509e-01 -4.55710813e-02
 -1.69742599e-01 -1.27954319e-01  1.14233634e-02  8.29739794e-02
 -1.65133014e-01 -1.26924366e-01 -3.18874381e-02 -1.16482250e-01
  2.97396518e-02 -7.62604177e-04  1.52792344e-02  6.18601218e-02
 -1.45620123e-01 -3.81449163e-01 -7.37554058e-02 -1.70712858e-01
  1.67583562e-02 -7.61220679e-02 -1.86123550e-02  4.80487794e-02
 -1.64117187e-01 -2.01446936e-02 -3.10911592e-02  4.96202819e-02
  3.01213060e-02 -1.91799216e-02  1.53848365e-01 -2.01187376e-03
 -5.37068248e-02 -1.54034168e-01  5.87601848e-02  2.94011414e-01
  1.46608874e-01  7.21874982e-02  2.20864397e-02  4.17589806e-02
  6.28009140e-02 -2.22463921e-01  6.87583536e-03  1.37114838e-01
  7.37359375e-02  3.10099926e-02  1.12748712e-01 -7.95104951e-02
  3.17516178e-02 -5.17255068e-02 -1.70513093e-01  4.05281857e-02
 -2.16966420e-02  1.13902

In [24]:
print('Processing unknown faces...')
# Now let's loop over a folder of faces we want to label
for filename in os.listdir(UNKNOWN_FACES_DIR):

    # Load image
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')

    # This time we first grab face locations - we'll need them to draw boxes
    locations = face_recognition.face_locations(image, model="cnn")

    # Now since we know loctions, we can pass them to face_encodings as second argument
    # Without that it will search for faces once again slowing down whole process
    encodings = face_recognition.face_encodings(image, locations)
    print(encodings[0].shape)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # But this time we assume that there might be more faces in an image - we can find faces of dirrerent people
    print(f', found {len(encodings)} face(s)')
    for face_encoding, face_location in zip(encodings, locations):

        # We use compare_faces (but might use face_distance as well)
        # Returns array of True/False values in order of passed known_faces
        results = face_recognition.compare_faces(known_faces, face_encoding,0.4)

        # Since order is being preserved, we check if any face was found then grab index
        # then label (name) of first matching known face withing a tolerance
        match = None
        if True in results:  # If at least one is true, get a name of first of found labels
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')

            # Each location contains positions in order: top, right, bottom, left
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])

            # Get color by name using our fancy function
            color = [0,255,0]

            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)

            # Now we need smaller, filled grame below for a name
            # This time we use bottom in both corners - to start from bottom and move 50 pixels down
            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22)

            # Paint frame
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)

            # Wite a name
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0,0), FONT_THICKNESS)

            # Show image
cv2.imshow(filename, image)
cv2.waitKey(0)
cv2.destroyWindow(filename)

Processing unknown faces...
Filename IMG_20200407_174629_063.jpg(128,)
, found 1 face(s)
 - tavva from [False, False, False, True, True]
Filename WhatsApp Image 2020-04-11 at 10.08.33 AM (1).jpeg(128,)
, found 1 face(s)
Filename WhatsApp Image 2020-04-11 at 10.08.33 AM.jpeg(128,)
, found 1 face(s)
 - ajay from [True, False, True, False, False]
